In [ ]:
#r "nuget:MyMediaLite"

In [ ]:
using System.IO;
using MyMediaLite.Data;

var ratings = File.ReadAllLines("data/ratings.csv")
    .Skip(1)
    .Select(line => line.Split(','))
    .Select(line => new { User = line[0].ToString(), Item = line[1].ToString(), Rating = Int32.Parse(line[2]) })
    .ToList();

var userIndex = new Dictionary<string, int>();
var itemIndex = new Dictionary<string, int>();

var userIDs = ratings.Select(r => r.User).Distinct();
var itemIDs = ratings.Select(r => r.Item).Distinct();

var userCount = 0;
var itemCount = 0;

foreach (var user in userIDs)
{
    userIndex.Add(user, userCount);
    userCount++;
}

foreach (var item in itemIDs)
{
    itemIndex.Add(item, itemCount);
    itemCount++;
}

In [ ]:
var training = new Ratings();
var test = new Ratings();

var random = new Random();

foreach (var rating in ratings)
{
    if (random.Next(0, 100) < 80)
    {
        training.Add(userIndex[rating.User], itemIndex[rating.Item], rating.Rating);
    }
    else
    {
        test.Add(userIndex[rating.User], itemIndex[rating.Item], rating.Rating);
    }
}

In [ ]:
Console.WriteLine(training.AllItems.Count());
Console.WriteLine(training.AllUsers.Count());

In [ ]:
using MyMediaLite.RatingPrediction;

var recommender = new SVDPlusPlus()
{
    Ratings = training,
    NumIter = 20,
    NumFactors = 20
};

recommender.Train();

In [ ]:
using MyMediaLite.Eval;

var results = recommender.Evaluate(test);

Console.WriteLine("RMSE={0} MAE={1}", results["RMSE"], results["MAE"]);
Console.WriteLine(results);